In [1]:
import numpy as np
from copy import deepcopy
from useful_structure import PartialLockedDict
from mathematical_science import MathModel
from check_and_response import ValueCheck
from Reaction import Reaction
import CoeffLaw

## Create a Reaction instance and play around

In [2]:
r = Reaction(
    reactants=dict(H=1,O2=1), 
    products=dict(OH=1,H=1),
    coeffLaw='modArrhenius', 
    coeffParams=dict(b=-1.0, E=-4.157)
)
r.get_params()

{'ID': 'reaction',
 'TYPE': 'Elementary',
 'coeffLaw': 'modArrhenius',
 'coeffParams': {'A': 1.0, 'E': -4.157, 'R': 8.314, 'b': -1.0},
 'coeffUnits': {},
 'products': {'H': 1, 'OH': 1},
 'reactants': {'H': 1, 'O2': 1},
 'reversible': False}

- #### Use `__str__` method to visualize the reaction equation.

In [3]:
print(r)

Reaction Equation:
1H + 1O2 => 1H + 1OH
----------------------------------------
Reaction Info:
ID: reaction
TYPE: Elementary
reversible: False
coeffLaw: modArrhenius
coeffParams: [('A', 1.0), ('E', -4.157), ('R', 8.314), ('b', -1.0)]
coeffUnits: []


## Compute right, Compute free

In [4]:
print(r.rateCoeff(check=False, T=0.5))
print(2 * np.e)

5.43656365691809
5.43656365691809


- #### We might want to apply some laws other than modified Arrhenius to compute the rate coeff. However, this will raise an error.

In [5]:
r.set_params(
    coeffLaw='NewLaw', 
    coeffParams=dict(E=4.157, R=8.314)
)
r.rateCoeff(check=True, T=-2.0)

NotImplementedError: coeffLaw = NewLaw. Referred reaction rate coefficient law is not implemented.

- #### Is there any approach to see what kind of laws are currently supported?

In [6]:
Reaction._CoeffLawDict.getcopy_all()

{'Arrhenius': CoeffLaw.Arrhenius,
 'Constant': CoeffLaw.Constant,
 'modArrhenius': CoeffLaw.modArrhenius}

- #### Is there any approach to design something new and add it to the programs "knowledge"?

In [7]:
class NewLaw(MathModel):
    
    @staticmethod
    def _kernel(T, E, R, **other_params):
        return -E / (T * R)
    
#     _check_np = ValueCheck(lambda x:x<=0.0, name='non-positive')
#     _check_zr = ValueCheck(lambda x:x==0.0, name='zero')
#     def check_stateparams(self, T, **other_params):
#         self._check_np.response(T, label='T', term='temperature')
#     @staticmethod
#     def check_coeffparams(R, **other_params):
#         NewLaw._check_zr.response(R, label='R', term='ideal gas constant')

Reaction._CoeffLawDict.update('NewLaw', NewLaw)
Reaction._CoeffLawDict.getcopy_all()

{'Arrhenius': CoeffLaw.Arrhenius,
 'Constant': CoeffLaw.Constant,
 'NewLaw': __main__.NewLaw,
 'modArrhenius': CoeffLaw.modArrhenius}

In [8]:
r.set_params(
    coeffLaw='NewLaw', 
    coeffParams=dict(E=4.157, R=8.314)
)
r.rateCoeff(check=True, T=-2.0)

0.25

- #### Good, but not enough. We also want some input check.

## What is a `MathModel`?

In [15]:
dir(MathModel)[-10:]

['__str__',
 '__subclasshook__',
 '__weakref__',
 '_default_settings',
 '_kernel',
 'check_coeffparams',
 'check_stateparams',
 'compute',
 'get_coeffparams',
 'get_defaults']

In [12]:
CoeffLaw.Arrhenius.get_defaults()

{'coeffparams': {'A': 1.0, 'E': 0.0, 'R': 8.314}, 'stateparams': {'T': 1e-16}}

## Structure of `_CoeffLawDict` (`PartialLockedDict`)

In [9]:
Reaction._CoeffLawDict.getcopy_builtin()

{'Arrhenius': CoeffLaw.Arrhenius,
 'Constant': CoeffLaw.Constant,
 'modArrhenius': CoeffLaw.modArrhenius}

In [10]:
Reaction._CoeffLawDict.reset()
Reaction._CoeffLawDict.getcopy_all()

{'Arrhenius': CoeffLaw.Arrhenius,
 'Constant': CoeffLaw.Constant,
 'modArrhenius': CoeffLaw.modArrhenius}

In [11]:
dir(Reaction._CoeffLawDict)[-15:]

['__str__',
 '__subclasshook__',
 '__weakref__',
 '_dict_all',
 '_dict_builtin',
 '_error_change_builtin',
 '_get_all',
 '_get_builtin',
 'getcopy',
 'getcopy_all',
 'getcopy_builtin',
 'remove',
 'reset',
 'update',
 'update_group']